<a href="https://colab.research.google.com/github/eswahn/northerngrounds/blob/main/Workshop_GAN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install StyleGAN2-ADA

In [ ]:
%tensorflow_version 1.x
!git clone https://github.com/eswahn/stylegan2-ada
%cd stylegan2-ada
!nvidia-smi -L

# Set training data
Set the correct path to your images below or upload them manually to /content/stylegan2-ada/images.

Mount Google Drive and set the correct output path if you want your results to be saved there.

Run the cell below when your settings are correct.

In [ ]:
#@markdown Paths
data_path = '/content/drive/MyDrive/Data/northerngrounds/axos.zip' #@param {type:"string"}
output_path = '/content/drive/MyDrive/Output/stylegan2-ada/' #@param {type:"string"}
#@markdown Image resolution
resolution = "512" #@param [512, 1024]
#@markdown If no previous checkpoint exists:
train_from = "wikiart model" #@param ["scratch", "wikiart model", "ffhq faces model"] {type:"string"}
#@markdown After how many ticks should model snapshots and output images be saved?
snapshot_count = 4 #@param {type:'integer'}
imgsnapshot_count =  1#@param {type:'integer'}
#@markdown Allow training data to be mirrored during training?
mirroredX = True #@param {type:"boolean"}
mirroredY = False #@param {type:"boolean"}

if train_from == 'wikiart model':
  if resolution == '512':
    !gdown --id 1HmJ2URt1WrTVvjQkhsAuR8I4UvmxreNH -O /content/wikiart.pkl
  if resolution == '1024':
    !gdown --id 1_GYAYJJLK_LebYZaEiSfHUWeU67VWeal -O /content/wikiart.pkl
  transfer_from = '/content/wikiart.pkl' # False: train from scratch, otherwise do tranfer learning

if train_from == 'ffhq face model':
  transfer_from = 'ffhq%i' % resolution

if train_from == 'scratch':
  transfer_from = False

!rm -rf /content/stylegan2-ada/images /content/stylegan2-ada/datasets
!mkdir /content/stylegan2-ada/images /content/stylegan2-ada/datasets

# Load training data

In [ ]:
import glob, os, pathlib

%cd /content/stylegan2-ada

dataset_name = os.path.basename(data_path)
images_path = '/content/stylegan2-ada/images/'

if '.' in dataset_name:
  dataset_name = dataset_name[:dataset_name.find('.')]

if os.path.exists(data_path):
  print('Using training set from %s' % data_path)
  %cd {images_path}
  if '.zip' in data_path:                       # zip archive
    !cp -a {data_path} {images_path}
    !unzip -qq *.zip
    !rm -rf *.zip
  elif '.tar.gz' in data_path:                  # .tar.gz archive
    !cp -a {data_path} {images_path}
    !tar -zxf *.tar.gz
    !rm -rf *.tar.gz
  else:                                         # directory
    !cp -a {data_path} {images_path}
#    images_path = data_path
else:
  print('Training set not found')


/content/stylegan2-ada
Using training set from /content/drive/MyDrive/Data/northerngrounds/axos.zip
/content/stylegan2-ada/images


In [ ]:
%cd /content/stylegan2-ada
!python dataset_tool.py create_from_images ./datasets/{dataset_name} images

# Train the model

If a previous checkpoint exists, continue training.

Otherwise, train from scratch or do transfer learning from a pretrained model.

In [ ]:
outdir = output_path + dataset_name + '/'

args = ''
if transfer_from:     # train from scratch
  network_pkls = glob.glob(os.path.join(outdir.replace('\ ',' '), '*%s*' % dataset_name, 'network-*.pkl'))
  if network_pkls:
    resume_from = sorted(network_pkls)[-1].replace(' ','\ ')
    nkimg = int(resume_from[-10:-4])
    args += ' --nkimg=%i' % nkimg
  else:
    resume_from = transfer_from
  args += ' --resume=%s' % resume_from
  print('Resuming training from %s' % resume_from)

args += ' --outdir %s' % outdir
args += ' --snap=%i' % snapshot_count
args += ' --imgsnap=%i' % imgsnapshot_count
args += ' --cfg=11gb-gpu'
args += ' --data=./datasets/%s' % dataset_name
args += ' --augpipe=bgcfnc' # was default = bgc
args += ' --mirror=%s' % str(mirroredX)
args += ' --mirrory=%s' % str(mirroredY)
args += ' --metrics=None'

!python train.py {args}